<a href="https://colab.research.google.com/github/TongleiChen/colab_notebook/blob/main/a4_surname_classifier_lstm_template_0403.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A4 - Surname Classifier

Author: Austin Blodgett

Adaptation to colab: Nitin Venkateswaran


### Follow the steps to use this notebook for your A4. 
**NOTE**: It is best to use your Georgetown Google accounts.
##### 1. Save a copy of this notebook starter template in your Google Drive (File -> Save a copy in drive)
##### 2. Upload a copy of the datafile files from **surname-data** directory (available in a4.zip) to your Google Drive in the location **A4/surname-data/surnames.csv**; you will need to create the folder 'A4' at the root location in your Drive, followed by the subfolder 'surname-data' 
##### 3. You are all set!



###Import libraries and mount Google Drive





In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import os, random
import tensorflow as tf

from collections import Counter
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.regularizers import l2

import numpy as np
from keras import backend as K



Mounted at /content/drive


In [ ]:
data_file = '/content/drive/My Drive/A4/surname-data/surnames.csv'
UNK = '[UNK]'
PAD = '[PAD]'
START = '<s>'
END = '</s>'

###Implement this function if you want to transform the input text, e.g. normalizing case

In [ ]:
# TODO
def transform_text_sequence(seq):
    '''
    Implement this function if you want to transform the input text,
    for example normalizing case.
    '''
    return seq

###Helper Functions (no need to implement)

In [ ]:
def get_vocabulary_and_data(data_file, split, max_vocab_size=None):
    vocab = Counter()
    data = []
    labels = []
    with open(data_file, 'r', encoding='utf8') as f:
        for line in f:
            cols = line.split(',')
            s, surname, label = cols[0].strip(), cols[1].strip(), cols[2].strip()
            if s==split:
                surname = list(surname)
                surname = [START]+surname+[END]
                data.append(transform_text_sequence(surname))
                labels.append(label)
            for tok in surname:
                vocab[tok]+=1

    vocab = sorted(vocab.keys(), key=lambda k: vocab[k], reverse=True)
    if max_vocab_size:
        vocab = vocab[:max_vocab_size-2]
    vocab = [UNK, PAD] + vocab

    return {k:v for v,k in enumerate(vocab)}, set(labels), data, labels


def vectorize_sequence(seq, vocab):
    seq = [tok if tok in vocab else UNK for tok in seq]
    return [vocab[tok] for tok in seq]


def unvectorize_sequence(seq, vocab):
    translate = sorted(vocab.keys(),key=lambda k:vocab[k])
    return [translate[i] for i in seq]


def one_hot_encode_label(label, label_set):
    vec = [1.0 if l==label else 0.0 for l in label_set]
    return np.array(vec)


def batch_generator(data, labels, vocab, label_set, batch_size=1):
    while True:
        batch_x = []
        batch_y = []
        for doc, label in zip(data,labels):
            batch_x.append(vectorize_sequence(doc, vocab))
            batch_y.append(one_hot_encode_label(label, label_set))
            if len(batch_x) >= batch_size:
                # Pad Sequences in batch to same length
                batch_x = pad_sequences(batch_x, vocab[PAD])
                yield np.array(batch_x), np.array(batch_y)
                batch_x = []
                batch_y = []


def describe_data(data, gold_labels, label_set, generator):
    batch_x, batch_y = [], []
    for bx, by in generator:
        batch_x = bx
        batch_y = by
        break
    print('Data example:',data[0])
    print('Label:',gold_labels[0])
    print('Label count:', len(label_set))
    print('Data size', len(data))
    print('Batch input shape:', batch_x.shape)
    print('Batch output shape:', batch_y.shape)


def pad_sequences(batch_x, pad_value):
    ''' This function should take a batch of sequences of different lengths
        and pad them with the pad_value token so that they are all the same length.

        Assume that batch_x is a list of lists.
    '''
    pad_length = len(max(batch_x, key=lambda x: len(x)))
    for i, x in enumerate(batch_x):
        if len(x) < pad_length:
            batch_x[i] = x + ([pad_value] * (pad_length - len(x)))

    return batch_x

###Change these arguments for the main procedure call as needed for your experiments

In [ ]:
epochs = 15 # number of epochs
learning_rate = 0.01 # learning rate
dropout = 0.3 # dropout rate
early_stopping = -1 # early stopping criteria
embedding_size = 100 # embedding dimension size
hidden_size = 40 # hidden layer size
batch_size = 50 # batch size

###Check the GPU is available

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  device_name = '/cpu:0'
  print(
      '\n\n This notebook is not '
      'configured to use a GPU.  You can change this in Notebook Settings. Defaulting to:' + device_name)
else:
  print ('GPU Device found: ' + device_name)

GPU Device found: /device:GPU:0


In [ ]:
vocab, labels, train_data, train_labels = get_vocabulary_and_data(data_file, 'train')
_, _, dev_data, dev_labels = get_vocabulary_and_data(data_file, 'dev')
_, _, test_data, test_labels = get_vocabulary_and_data(data_file, 'test')

describe_data(train_data, train_labels, labels,
                  batch_generator(train_data, train_labels, vocab, labels, batch_size))

Data example: ['<s>', 'H', 'a', 'd', 'a', 'd', '</s>']
Label: arabic
Label count: 19
Data size 15000
Batch input shape: (50, 14)
Batch output shape: (50, 19)


In [ ]:
len(vocab)

91

###Main procedure call: Implement the keras model here

##### Use the variables batch_size, hidden_size, embedding_size, dropout, epochs here.

In [ ]:
vocab, labels, train_data, train_labels = get_vocabulary_and_data(data_file, 'train')
_, _, dev_data, dev_labels = get_vocabulary_and_data(data_file, 'dev')
_, _, test_data, test_labels = get_vocabulary_and_data(data_file, 'test')

describe_data(train_data, train_labels, labels,
                  batch_generator(train_data, train_labels, vocab, labels, batch_size))

with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Data example: ['<s>', 'H', 'a', 'd', 'a', 'd', '</s>']
Label: arabic
Label count: 19
Data size 15000
Batch input shape: (50, 14)
Batch output shape: (50, 19)
Epoch 1 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 1.0513 - accuracy: 0.6804
Dev Loss: 1.0513139963150024 Dev Acc: 0.6803921461105347
Epoch 2 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 0.9270 - accuracy: 0.7121
Dev Loss: 0.9270397424697876 Dev Acc: 0.7120915055274963
Epoch 3 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8986 - accuracy: 0.7314
Dev Loss: 0.8985798954963684 Dev Acc: 0.7313725352287292
Epoch 4 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8560 - accuracy: 0.7425
Dev Loss: 0.8560300469398499 Dev Acc: 0.7424836754798889
Epoch 5 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8485 - accuracy: 0.7490
Dev Loss: 0.8485093712806702 Dev Acc: 0.7490196228027344
Epoch 6 / 10
3060/3060 [========

In [ ]:
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 15
3060/3060 [==============================] - 13s 4ms/step - loss: 1.0541 - accuracy: 0.6807
Dev Loss: 1.0541378259658813 Dev Acc: 0.6807189583778381
Epoch 2 / 15
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8359 - accuracy: 0.7582
Dev Loss: 0.8359115123748779 Dev Acc: 0.758169949054718
Epoch 3 / 15
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7668 - accuracy: 0.7739
Dev Loss: 0.7668155431747437 Dev Acc: 0.7738562226295471
Epoch 4 / 15
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7777 - accuracy: 0.7742
Dev Loss: 0.7777050733566284 Dev Acc: 0.7741830348968506
Epoch 5 / 15
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7815 - accuracy: 0.7781
Dev Loss: 0.7815014719963074 Dev Acc: 0.7781046032905579
Epoch 6 / 15
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8095 - accuracy: 0.7732
Dev Loss: 0.8094638586044312 Dev Acc: 0.7732025980949402
Epoch 7 / 15
3060/3060 [=====

In [ ]:
# test evaluation 
loss, acc = classifier.evaluate(batch_generator(test_data, test_labels, vocab, labels),
                                                  steps=len(test_data))
print('Test Loss:', loss, 'Test Acc:', acc)

2014/2014 [==============================] - 8s 4ms/step - loss: 0.8795 - accuracy: 0.7736
Test Loss: 0.8794943690299988 Test Acc: 0.7735849022865295


In [ ]:
embedding_size = 300
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 10
3060/3060 [==============================] - 14s 4ms/step - loss: 0.9762 - accuracy: 0.7118
Dev Loss: 0.9762228727340698 Dev Acc: 0.7117646932601929
Epoch 2 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8610 - accuracy: 0.7503
Dev Loss: 0.8609779477119446 Dev Acc: 0.7503268122673035
Epoch 3 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8259 - accuracy: 0.7516
Dev Loss: 0.8259142637252808 Dev Acc: 0.7516340017318726
Epoch 4 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8497 - accuracy: 0.7435
Dev Loss: 0.8497036695480347 Dev Acc: 0.7434640526771545
Epoch 5 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8202 - accuracy: 0.7647
Dev Loss: 0.8201887607574463 Dev Acc: 0.7647058963775635
Epoch 6 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8267 - accuracy: 0.7608
Dev Loss: 0.8267378807067871 Dev Acc: 0.7607843279838562
Epoch 7 / 10
3060/3060 [====

In [ ]:
embedding_size = 50
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 1.1235 - accuracy: 0.6680
Dev Loss: 1.1234782934188843 Dev Acc: 0.6679738759994507
Epoch 2 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8222 - accuracy: 0.7503
Dev Loss: 0.8221744894981384 Dev Acc: 0.7503268122673035
Epoch 3 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7469 - accuracy: 0.7833
Dev Loss: 0.7468716502189636 Dev Acc: 0.7833333611488342
Epoch 4 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7562 - accuracy: 0.7761
Dev Loss: 0.7561596035957336 Dev Acc: 0.7761437892913818
Epoch 5 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7775 - accuracy: 0.7778
Dev Loss: 0.7774919867515564 Dev Acc: 0.7777777910232544
Epoch 6 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8074 - accuracy: 0.7755
Dev Loss: 0.8074012398719788 Dev Acc: 0.7754902243614197
Epoch 7 / 10
3060/3060 [====

In [ ]:
hidden_size = 30
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 15
3060/3060 [==============================] - 15s 5ms/step - loss: 1.1575 - accuracy: 0.6598
Dev Loss: 1.1574530601501465 Dev Acc: 0.6598039269447327
Epoch 2 / 15
3060/3060 [==============================] - 13s 4ms/step - loss: 0.9292 - accuracy: 0.7304
Dev Loss: 0.9291821718215942 Dev Acc: 0.7303921580314636
Epoch 3 / 15
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8616 - accuracy: 0.7503
Dev Loss: 0.8615626096725464 Dev Acc: 0.7503268122673035
Epoch 4 / 15
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8531 - accuracy: 0.7588
Dev Loss: 0.8531039953231812 Dev Acc: 0.7588235139846802
Epoch 5 / 15
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8435 - accuracy: 0.7595
Dev Loss: 0.8435284495353699 Dev Acc: 0.7594771385192871
Epoch 6 / 15
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8440 - accuracy: 0.7676
Dev Loss: 0.8440449833869934 Dev Acc: 0.7676470875740051
Epoch 7 / 15
3060/3060 [====

In [ ]:
hidden_size = 20
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 1.1121 - accuracy: 0.6905
Dev Loss: 1.1121164560317993 Dev Acc: 0.6905228495597839
Epoch 2 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 0.9851 - accuracy: 0.7137
Dev Loss: 0.9851279258728027 Dev Acc: 0.7137255072593689
Epoch 3 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8797 - accuracy: 0.7310
Dev Loss: 0.8797189593315125 Dev Acc: 0.7310457229614258
Epoch 4 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8448 - accuracy: 0.7418
Dev Loss: 0.8448327779769897 Dev Acc: 0.741830050945282
Epoch 5 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8373 - accuracy: 0.7471
Dev Loss: 0.8373358845710754 Dev Acc: 0.7470588088035583
Epoch 6 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8354 - accuracy: 0.7490
Dev Loss: 0.8354207873344421 Dev Acc: 0.7490196228027344
Epoch 7 / 10
3060/3060 [=====

In [ ]:
hidden_size = 50
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 0.9868 - accuracy: 0.7160
Dev Loss: 0.9867905378341675 Dev Acc: 0.7160130739212036
Epoch 2 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8663 - accuracy: 0.7405
Dev Loss: 0.8662500381469727 Dev Acc: 0.7405228614807129
Epoch 3 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7619 - accuracy: 0.7807
Dev Loss: 0.7619409561157227 Dev Acc: 0.780718982219696
Epoch 4 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7714 - accuracy: 0.7843
Dev Loss: 0.7713531255722046 Dev Acc: 0.7843137383460999
Epoch 5 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7657 - accuracy: 0.7869
Dev Loss: 0.7657081484794617 Dev Acc: 0.786928117275238
Epoch 6 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8102 - accuracy: 0.7742
Dev Loss: 0.8102098107337952 Dev Acc: 0.7741830348968506
Epoch 7 / 10
3060/3060 [======

In [ ]:
hidden_size = 10
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 10
3060/3060 [==============================] - 14s 4ms/step - loss: 1.2415 - accuracy: 0.6552
Dev Loss: 1.2415069341659546 Dev Acc: 0.6552287340164185
Epoch 2 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 1.1990 - accuracy: 0.6503
Dev Loss: 1.1990325450897217 Dev Acc: 0.6503267884254456
Epoch 3 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 1.1019 - accuracy: 0.6784
Dev Loss: 1.1019408702850342 Dev Acc: 0.6784313917160034
Epoch 4 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 1.0540 - accuracy: 0.7000
Dev Loss: 1.0539747476577759 Dev Acc: 0.699999988079071
Epoch 5 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 1.0112 - accuracy: 0.7193
Dev Loss: 1.011244535446167 Dev Acc: 0.719281017780304
Epoch 6 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 1.0511 - accuracy: 0.7042
Dev Loss: 1.0510516166687012 Dev Acc: 0.7042483687400818
Epoch 7 / 10
3060/3060 [=======

In [ ]:
hidden_size = 60
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 10
3060/3060 [==============================] - 13s 4ms/step - loss: 1.0103 - accuracy: 0.6984
Dev Loss: 1.0102802515029907 Dev Acc: 0.6983659863471985
Epoch 2 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7424 - accuracy: 0.7765
Dev Loss: 0.7423787713050842 Dev Acc: 0.7764706015586853
Epoch 3 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7140 - accuracy: 0.7833
Dev Loss: 0.7139737010002136 Dev Acc: 0.7833333611488342
Epoch 4 / 10
3060/3060 [==============================] - 11s 4ms/step - loss: 0.7314 - accuracy: 0.7886
Dev Loss: 0.7313960790634155 Dev Acc: 0.7885621190071106
Epoch 5 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7375 - accuracy: 0.7902
Dev Loss: 0.7374964356422424 Dev Acc: 0.7901960611343384
Epoch 6 / 10
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8051 - accuracy: 0.7797
Dev Loss: 0.8051390647888184 Dev Acc: 0.7797385454177856
Epoch 7 / 10
3060/3060 [====

In [ ]:
hidden_size = 50
embedding_size = 300
epochs = 8
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 8
3060/3060 [==============================] - 16s 5ms/step - loss: 1.0146 - accuracy: 0.6840
Dev Loss: 1.0146058797836304 Dev Acc: 0.6839869022369385
Epoch 2 / 8
3060/3060 [==============================] - 12s 4ms/step - loss: 0.9735 - accuracy: 0.6990
Dev Loss: 0.9734741449356079 Dev Acc: 0.6990196108818054
Epoch 3 / 8
3060/3060 [==============================] - 14s 5ms/step - loss: 0.8952 - accuracy: 0.7314
Dev Loss: 0.895188570022583 Dev Acc: 0.7313725352287292
Epoch 4 / 8
3060/3060 [==============================] - 12s 4ms/step - loss: 0.8396 - accuracy: 0.7526
Dev Loss: 0.8395711779594421 Dev Acc: 0.7526143789291382
Epoch 5 / 8
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8439 - accuracy: 0.7503
Dev Loss: 0.8439297676086426 Dev Acc: 0.7503268122673035
Epoch 6 / 8
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8606 - accuracy: 0.7487
Dev Loss: 0.860573947429657 Dev Acc: 0.7486928105354309
Epoch 7 / 8
3060/3060 [=============

In [ ]:
hidden_size = 50
embedding_size = 150
epochs = 8
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 8
3060/3060 [==============================] - 15s 5ms/step - loss: 0.9532 - accuracy: 0.7069
Dev Loss: 0.9532268643379211 Dev Acc: 0.70686274766922
Epoch 2 / 8
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7967 - accuracy: 0.7699
Dev Loss: 0.7966799139976501 Dev Acc: 0.7699346542358398
Epoch 3 / 8
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7742 - accuracy: 0.7725
Dev Loss: 0.7741916179656982 Dev Acc: 0.772549033164978
Epoch 4 / 8
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7495 - accuracy: 0.7775
Dev Loss: 0.7494708895683289 Dev Acc: 0.7774509787559509
Epoch 5 / 8
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7675 - accuracy: 0.7846
Dev Loss: 0.7674776911735535 Dev Acc: 0.7846405506134033
Epoch 6 / 8
3060/3060 [==============================] - 11s 4ms/step - loss: 0.7811 - accuracy: 0.7791
Dev Loss: 0.7811238765716553 Dev Acc: 0.7790849804878235
Epoch 7 / 8
3060/3060 [==============

In [ ]:
hidden_size = 50
embedding_size = 50
epochs = 8
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 8
3060/3060 [==============================] - 13s 4ms/step - loss: 1.0005 - accuracy: 0.7052
Dev Loss: 1.000499963760376 Dev Acc: 0.7052287459373474
Epoch 2 / 8
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8405 - accuracy: 0.7487
Dev Loss: 0.8405497074127197 Dev Acc: 0.7486928105354309
Epoch 3 / 8
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7820 - accuracy: 0.7719
Dev Loss: 0.7820004820823669 Dev Acc: 0.7718954086303711
Epoch 4 / 8
3060/3060 [==============================] - 17s 6ms/step - loss: 0.7696 - accuracy: 0.7690
Dev Loss: 0.769624650478363 Dev Acc: 0.7689542770385742
Epoch 5 / 8
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7729 - accuracy: 0.7797
Dev Loss: 0.7728553414344788 Dev Acc: 0.7797385454177856
Epoch 6 / 8
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7980 - accuracy: 0.7761
Dev Loss: 0.7979559302330017 Dev Acc: 0.7761437892913818
Epoch 7 / 8
3060/3060 [=============

In [ ]:
hidden_size = 50
embedding_size = 100
epochs = 5
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    classifier = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(labels)
    classifier.add(tf.keras.layers.Embedding(input_size, embedding_size, input_length=batch_size))
    classifier.add(Bidirectional(LSTM(hidden_size, return_sequences=False)))
    classifier.add(tf.keras.layers.Dense(output_size, activation='softmax'))
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    # ------------------------------------------------------------------------------------------------

    classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        classifier.fit(batch_generator(train_data, train_labels, vocab, labels, batch_size=batch_size),
                                  epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc = classifier.evaluate(batch_generator(dev_data, dev_labels, vocab, labels),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc)


Epoch 1 / 5
3060/3060 [==============================] - 13s 4ms/step - loss: 1.0044 - accuracy: 0.7085
Dev Loss: 1.004434585571289 Dev Acc: 0.7084967494010925
Epoch 2 / 5
3060/3060 [==============================] - 13s 4ms/step - loss: 0.8303 - accuracy: 0.7539
Dev Loss: 0.8302643299102783 Dev Acc: 0.7539215683937073
Epoch 3 / 5
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7628 - accuracy: 0.7752
Dev Loss: 0.7627842426300049 Dev Acc: 0.7751634120941162
Epoch 4 / 5
3060/3060 [==============================] - 12s 4ms/step - loss: 0.7540 - accuracy: 0.7866
Dev Loss: 0.7540210485458374 Dev Acc: 0.7866013050079346
Epoch 5 / 5
3060/3060 [==============================] - 13s 4ms/step - loss: 0.7622 - accuracy: 0.7820
Dev Loss: 0.7621514797210693 Dev Acc: 0.7820261716842651


In [ ]:
loss, acc = classifier.evaluate(batch_generator(test_data, test_labels, vocab, labels),
                                                  steps=len(test_data))
print('Test Loss:', loss, 'Test Acc:', acc)

2014/2014 [==============================] - 9s 4ms/step - loss: 0.7450 - accuracy: 0.7835
Test Loss: 0.7449893355369568 Test Acc: 0.7835153937339783
